# 02 - Preprocessing Reviews (Optional)

> **Note:** Preprocessing is now integrated into `01_scrape_reviews.ipynb`. The scraper outputs already-cleaned data with `content_clean` and `language` columns, and saves processed files directly to `data/processed/`.
>
> This notebook is kept as a **reference/standalone tool** — useful if you need to re-process raw data separately without re-scraping.

Clean and prepare scraped reviews:
- Remove nulls and duplicates
- Clean text (whitespace, special characters, unicode)
- Detect language and optionally translate non-English reviews

In [ ]:
import pandas as pd
import re
import unicodedata
from datetime import datetime
from langdetect import detect, LangDetectException
import os

In [ ]:
# Load raw data — update filename to match the scraper output date tag
INPUT_FILE = "data/raw/all_reviews.csv"  # Update with date tag, e.g. all_reviews_20260217.csv
OUTPUT_DIR = "data/processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Version/date tag for output files
RUN_DATE = datetime.now().strftime("%Y%m%d")

# Auto-detect latest raw file if default doesn't exist
if not os.path.exists(INPUT_FILE):
    import glob
    raw_files = sorted(glob.glob("data/raw/all_reviews_*.csv"))
    if raw_files:
        INPUT_FILE = raw_files[-1]
        print(f"Using latest raw file: {INPUT_FILE}")

df = pd.read_csv(INPUT_FILE, parse_dates=["at"])
print(f"Loaded {len(df)} reviews from {INPUT_FILE}")
print(f"Output file tag: {RUN_DATE}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
# Check data quality before cleaning
print("=" * 40)
print("DATA QUALITY CHECK")
print("=" * 40)
print(f"Total rows: {len(df)}")
print(f"Null content: {df['content'].isna().sum()}")
print(f"Empty content: {(df['content'].str.strip() == '').sum() if df['content'].notna().any() else 0}")
print(f"Duplicate reviewIds: {df['reviewId'].duplicated().sum()}")
print(f"\nReviews per app:")
print(df['app_name'].value_counts())

In [ ]:
def clean_text(text):
    """Clean review text."""
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # Normalize unicode characters
    text = unicodedata.normalize("NFKD", text)
    
    # Replace newlines and tabs with space
    text = re.sub(r"[\n\r\t]+", " ", text)
    
    # Remove emojis and special unicode symbols (keep basic punctuation)
    text = re.sub(
        r"[\U00010000-\U0010ffff]",  # Supplementary Unicode planes (emojis etc)
        "",
        text,
    )
    
    # Remove excessive punctuation repetition (e.g., "!!!!!!" -> "!")
    text = re.sub(r"([!?.])\1{2,}", r"\1", text)
    
    # Collapse multiple spaces
    text = re.sub(r"\s{2,}", " ", text)
    
    return text.strip()

In [ ]:
# Step 1: Remove duplicates
before = len(df)
df = df.drop_duplicates(subset="reviewId")
print(f"Removed {before - len(df)} duplicates. Remaining: {len(df)}")

# Step 2: Remove null/empty content
df = df.dropna(subset=["content"])
df = df[df["content"].str.strip() != ""]
print(f"After removing empty reviews: {len(df)}")

# Step 3: Clean text
df["content_clean"] = df["content"].apply(clean_text)

# Remove rows where cleaning resulted in empty text
df = df[df["content_clean"].str.len() > 0]
print(f"After text cleaning: {len(df)}")

In [ ]:
# Step 4: Detect language
def detect_language(text):
    """Detect language of text. Returns language code or 'unknown'."""
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

print("Detecting languages...")
df["language"] = df["content_clean"].apply(detect_language)

print(f"\nLanguage distribution (top 10):")
print(df["language"].value_counts().head(10))

en_count = (df["language"] == "en").sum()
print(f"\nEnglish reviews: {en_count} ({en_count/len(df)*100:.1f}%)")

In [ ]:
# Step 5: Save processed data
output_file = f"{OUTPUT_DIR}/reviews_cleaned_{RUN_DATE}.csv"
df.to_csv(output_file, index=False)
print(f"Saved {len(df)} cleaned reviews to {output_file}")

# Also save English-only subset for sentiment analysis
df_en = df[df["language"] == "en"]
en_file = f"{OUTPUT_DIR}/reviews_english_{RUN_DATE}.csv"
df_en.to_csv(en_file, index=False)
print(f"Saved {len(df_en)} English reviews to {en_file}")

In [ ]:
# Summary
print("\n" + "="*50)
print("PREPROCESSING SUMMARY")
print("="*50)
print(f"Total cleaned reviews: {len(df)}")
print(f"English reviews: {len(df_en)}")
print(f"\nPer app:")
for app in df["app_name"].unique():
    total = len(df[df["app_name"] == app])
    en = len(df_en[df_en["app_name"] == app])
    print(f"  {app}: {total} total, {en} English")

# Show sample cleaned reviews
print(f"\nSample cleaned reviews:")
df[["app_name", "content_clean", "score", "language"]].sample(5)